In [47]:
import sys
sys.path.append('/media/rene/LLR/lib/python3.6/site-packages')
import numpy as np
import matplotlib.pyplot as plt
import cma
from es import CMAES

In [48]:
def function_banana(inp):
    (x, y) = inp[0], inp[1]
    val = (1 - x)**2 + 100*(y - x**2)**2
    return val

fit_func = function_banana

In [44]:
NPARAMS = 2        # 2 dim
NPOPULATION = 10    # use population size of 10
MAX_ITERATION = 200 # run each solver for 500 generations.

In [45]:
# defines a function to use solver to solve fit_func
def test_solver(solver, fit_func):
    history = []
    for j in range(MAX_ITERATION):
        solutions = solver.ask()
        fitness_list = np.zeros(solver.popsize)
        for i in range(solver.popsize):
            fitness_list[i] = fit_func(solutions[i])
        solver.tell(fitness_list)
        result = solver.result() # first element is the best solution, second element is the best fitness
        history.append(result[1])
        if (j+1) % 100 == 0:
            print("fitness at iteration", (j+1), result[1])
    print("local optimum discovered by solver:\n", result[0])
    print("fitness score at this local optimum:", result[1])
    return history

In [46]:
cmaes = CMAES(NPARAMS,
              popsize=NPOPULATION,
              weight_decay=0.0,
              sigma_init = 0.5
          )

cma_history = test_solver(cmaes, function_banana)

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 2 (seed=603697, Thu Oct 25 23:03:17 2018)
WARNING (class=CMAAdaptSigmaCSA method=update iteration=9):  sigma change np.exp(1.0011080358221212) = 2.7212954513923515 clipped to np.exp(+-1)
WARNING (class=CMAAdaptSigmaCSA method=update iteration=50):  sigma change np.exp(1.0069986306977963) = 2.737372806537307 clipped to np.exp(+-1)
WARNING (class=CMAAdaptSigmaCSA method=update iteration=51):  sigma change np.exp(1.054439927473229) = 2.870367089878352 clipped to np.exp(+-1)
fitness at iteration 100 2.617132473501196e+102
WARNING (class=CMAAdaptSigmaCSA method=update iteration=110):  sigma change np.exp(1.0544996344622353) = 2.8705384759710544 clipped to np.exp(+-1)
WARNING (class=CMAAdaptSigmaCSA method=update iteration=164):  sigma change np.exp(1.0200840800927098) = 2.7734279442399266 clipped to np.exp(+-1)
WARNING (class=CMAAdaptSigmaCSA method=update iteration=195):  sigma change np.exp(1.038919272705942) = 2.826161053930694 clipped to 

In [49]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import display, HTML
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Nicer way to import the module?
sys.path.append(str(Path.cwd().parent))

import torch
from torch import nn
import torch.optim as optim
from torch.optim import lr_scheduler
import cma
from es import CMAES

from utils.data import make_generators_DF_cifar, make_generators_DF_MNIST
from utils.loading import net_from_args
from utils.scheduler import ListScheduler
from utils.train_val import train_net

/home/rene/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [50]:
def main(args):
    batch_size, num_workers = int(args.batch_size), int(args.num_workers)
    IM_SIZE = int(args.IM_SIZE)
    device = torch.device(args.device)

    NPARAMS = int(args.epochs)  # one learning rate (parameter) per epoch
    NPOPULATION = int(args.NPOPULATION)  # population size
    MAX_GENERATIONS = int(args.MAX_GENERATIONS)  # number of generations

    SAVE_PATH = Path(args.SAVE_PATH)
    SAVE_PATH.mkdir(parents=True, exist_ok=True)
    with open(args.files_df_loc, 'rb') as f:
        files_df = pickle.load(f)

    # Use CMA-ES to evolve:
    solver = CMAES(NPARAMS,
                  popsize=NPOPULATION,
                  weight_decay=0.0,
                  sigma_init = 0.5
              )
    # ??? maybe make history much more extensive
    history = []
    for j in range(MAX_GENERATIONS):
        solutions = solver.ask()
        fitness_list = np.zeros(solver.popsize)

         # ??? Make generators for each generation. Does this matter? or just do it once???
        if args.dataset == 'CIFAR10':
            dataloaders = make_generators_DF_cifar(files_df, batch_size, num_workers, size=IM_SIZE, 
                                                    path_colname='path', adv_path_colname=None, return_loc=False)
        elif args.dataset == 'MNIST':
            dataloaders = make_generators_DF_MNIST(files_df, batch_size, num_workers, size=IM_SIZE,
                                                    path_colname='path', adv_path_colname=None, return_loc=False, normalize=True)

        # evaluate each set of learning rates, using new model each time:
        for i in range(solver.popsize):
            model, model_name = net_from_args(args, num_classes=10, IM_SIZE=IM_SIZE)
            model = model.to(device)

            # Train it for the given lr list:
            metrics = train_net(model, dataloaders, all_lr_lists[i], batch_size)
            # the fitness is the best validation accuracy:
            fitness_list[i] = metrics['best_val_acc']
        solver.tell(fitness_list)
        result = solver.result() # first element is the best solution, second element is the best fitness
        history.append(result[1])
        print("fitness at generation", (j+1), result[1])
    print("local optimum discovered by solver:\n", result[0])
    print("fitness score at this local optimum:", result[1])
        
    pickle.dump(history, open(str(SAVE_PATH)+'/'+str(model_name)+'_nGen_'+str(MAX_GENERATIONS)+'_nPop_'+str(NPOPULATION)+'_metrics.pkl', "wb"))

In [51]:
args = {}

args['files_df_loc'] = '/media/rene/data/learn-lr/MNIST/files_df.pkl'
args['SAVE_PATH'] = '/media/rene/data/learn-lr/MNIST/models'
args['net_type'] = 'SimpleNetMNIST'
args['depth'] = '18'
args['batch_size'] = 512
args['device'] = 'cuda:1'
args['dataset'] = 'MNIST'
args['IM_SIZE'] = 28
args['MAX_GENERATIONS'] = 5
args['NPOPULATION'] = 10


with torch.cuda.device(torch.device(args.device).index):
    main(args)

AttributeError: 'dict' object has no attribute 'device'